In [1]:
import pandas as pd
import numpy as np
import yfinance
from pycaret import regression

model = regression.load_model("saved_model")

Transformation Pipeline and Model Successfully Loaded


In [2]:
stock = "TSLA"

In [3]:
data = yfinance.download(stock,period="2y")
data = data["Adj Close"]
data = data.reset_index()
data["Date"] = pd.to_datetime(data["Date"])
data["Day"] = [i.day for i in data["Date"]]
data["Month"] = [i.month for i in data["Date"]]
data["Year"] = [i.year for i in data["Date"]]
data["Series"] = np.arange(1,len(data)+1)
data.drop("Date",axis=1,inplace=True,errors="ignore")
data = data[["Series","Year","Month","Day","Adj Close"]]
data

[*********************100%***********************]  1 of 1 completed


,Series,Year,Month,Day,Adj Close
0,1,2021,1,5,245.036667
1,2,2021,1,6,251.993332
2,3,2021,1,7,272.013336
3,4,2021,1,8,293.339996
4,5,2021,1,11,270.396667
...,...,...,...,...,...
499,500,2022,12,28,112.709999
500,501,2022,12,29,121.820000
501,502,2022,12,30,123.180000
502,503,2023,1,3,108.099998


In [4]:
model

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression',
                                      numerical_features=['Year', 'Month',
                                                          'Day', 'Series'],
                                      target='Adj Close')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catag...
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                              

In [5]:
future_dates = pd.date_range(start = '2023-01-05', end = '2025-01-01', freq = 'B')
future_df = pd.DataFrame()
future_df["Day"] = [i.day for i in future_dates]
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates] 
future_df['Series'] = np.arange(15358,(15358+len(future_dates)))
future_df

,Day,Month,Year,Series
0,5,1,2023,15358
1,6,1,2023,15359
2,9,1,2023,15360
3,10,1,2023,15361
4,11,1,2023,15362
...,...,...,...,...
515,26,12,2024,15873
516,27,12,2024,15874
517,30,12,2024,15875
518,31,12,2024,15876


In [6]:
predictions_future = regression.predict_model(model,data=future_df)
predictions_future

,Day,Month,Year,Series,Label
0,5,1,2023,15358,244.902329
1,6,1,2023,15359,244.217545
2,9,1,2023,15360,244.017208
3,10,1,2023,15361,244.088839
4,11,1,2023,15362,244.387957
...,...,...,...,...,...
515,26,12,2024,15873,238.380730
516,27,12,2024,15874,238.155230
517,30,12,2024,15875,240.783033
518,31,12,2024,15876,240.696647


In [7]:
import plotly.express as px
concat_df = pd.concat([data,predictions_future], axis=0)
concat_df['Date'] = pd.to_datetime(dict(year=concat_df["Year"], month=concat_df["Month"], day=concat_df["Day"]))
concat_df.set_index(concat_df["Date"],inplace=True)
concat_df.drop("Date",axis=1,inplace=True,errors="ignore")
fig = px.line(concat_df, x=concat_df.index, y=["Adj Close", "Label"], template = 'plotly_dark')
fig.show()